# Creating Overview of the Mosaic


### Requirement
```
!pip install tdqm cogeo-mosaic==3.0a3 rio-cogeo
```

![](https://user-images.githubusercontent.com/10407788/81730358-65dae300-945b-11ea-9522-b487f9b3e244.png)

In [1]:
import time
import random
import requests
import urllib.parse
from io import BytesIO
from concurrent import futures

from tqdm.notebook import tqdm

import mercantile
from supermercado.burntiles import tile_extrema
from affine import Affine

import rasterio
from rasterio.io import MemoryFile
from rasterio.windows import Window
from rasterio.enums import ColorInterp

from rio_cogeo.utils import _meters_per_pixel
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

from rio_tiler_mosaic.mosaic import _filter_futures

In [2]:
mosaicid = "ab2ea71fc83c6f70461a71c5da4b3d25378e678099d88bd6ff53240f"
endpoint = "https://mosaic.cogeo.xyz"

In [7]:
# Fetch the mosaic info
info = requests.get(f"{endpoint}/{mosaicid}/info").json()
base_zoom = info["minzoom"] - 1
bounds = info["bounds"]

# Find all the tiles we need to fetch to contruct the overview
extrema = tile_extrema(bounds, base_zoom)
tilesize = 256
res = _meters_per_pixel(base_zoom, 0, tilesize=tilesize)

tiles = []
for x in range(extrema["x"]["min"], extrema["x"]["max"]):
    for y in range(extrema["y"]["min"], extrema["y"]["max"]):
        tiles.append(mercantile.Tile(z=base_zoom, x=x, y=y))

# shuffle the tiles makes the progressbar happy
random.shuffle(tiles)

In [8]:
# Here we define the tile URL. 
# using .tif means we will return the a GeoTIFF with all the band + a mask
# our intput data was RGB so the TIFF will be RGB+Alpha
tile_endpoint = f"https://mosaic.cogeo.xyz/{mosaicid}/{{z}}/{{x}}/{{y}}.tif"

In [9]:
# Define Output COG parameters
width = (extrema["x"]["max"] - extrema["x"]["min"]) * tilesize
height = (extrema["y"]["max"] - extrema["y"]["min"]) * tilesize
w, n = mercantile.xy(
    *mercantile.ul(extrema["x"]["min"], extrema["y"]["min"], base_zoom)
)

params = dict(
    driver="GTiff",
    dtype="uint8",  # We know our files are uint8
    count=4,  # RGB + ALPHA
    width=width,
    height=height,
    crs="epsg:3857",
    transform=Affine(res, 0, w, 0, -res, n),
    tiled=True,
    blockxsize=256,
    blockysize=256,
)

output_profile = cog_profiles.get("jpeg") # if the files aren't uint8, this will need to be changed
output_profile["blockxsize"] = 256
output_profile["blockysize"] = 256

In [11]:
def _worker(tile, retry=0):
    """
    Tile Worker.
    
    Call the tile endpoint for each mercator tile.
    
    """
    url = tile_endpoint.format(z=tile.z, x=tile.x, y=tile.y)  # populate tile url
    
    # Sometime the tiler can fail but we can retry it
    img = requests.get(url)
    if not img.status_code in [200, 202]:
        time.sleep(3)
        if retry == 3:
            raise Exception("Empty")
        return _worker(tile, retry=retry+1)
    
    # Here we calculate the output window for the tile
    row = (tile.y - extrema["y"]["min"]) * tilesize
    col = (tile.x - extrema["x"]["min"]) * tilesize
    window = Window(col_off=col, row_off=row, width=tilesize, height=tilesize)

    return window, img

In [12]:
# We create a first file in memory where to write all the `tiles` and then we create a COG from it
with MemoryFile() as memfile:
    with memfile.open(**params) as mem:

        with futures.ThreadPoolExecutor(max_workers=50) as executor:
            future_work = [
                executor.submit(_worker, tile) for tile in tiles
            ]

            for f in tqdm(futures.as_completed(future_work), total=len(future_work)):               
                pass

        for f in _filter_futures(future_work):
            window, img = f
            with rasterio.open(BytesIO(img.content)) as src_dst:
                mem.write(src_dst.read(), window=window)
        
        # We set colorinterp to RGBA
        mem.colorinterp = [ColorInterp.red, ColorInterp.green, ColorInterp.blue, ColorInterp.alpha]

    cog_translate(
        memfile,
        f"Padang.tif",
        output_profile,
        indexes=(1,2,3), # we remove the alpha band and forward it to a mask band (with add_mask=True)
        add_mask=True,
        config=dict(GDAL_NUM_THREADS="ALL_CPUS", GDAL_TIFF_OVR_BLOCKSIZE="128", GDAL_TIFF_INTERNAL_MASK="TRUE"),
    )

Reading input: <rasterio.io.MemoryFile object at 0x110a9e850>
Adding overviews...
Updating dataset tags...
Writing output to: Padang.tif
